# Spiking Neural Networks for Land Cover and Land Use Classification
This notebook is a tutorial on how to reproduce the results from [A. Kucik, G. Meoni, *Investigating Spiking Neural Networks for Energy-Efficient On-Board AI Applications. A Case Study in Land Cover and Land Use Classification
*](https://ieeexplore.ieee.org/document/9522999). It will guide you through artificial neural network (ANN) training, spi

## Requirements
This notebook uses [Python](https://www.python.org/) 3.10 and requires the following third-party libraries libraries (with their recommended versions indicated):
* [KerasSpiking](https://www.nengo.ai/keras-spiking/) 0.3.0
* [NumPy](https://numpy.org/) 1.19.5
* [TensorFlow](https://www.tensorflow.org/) 2.6.2
* [TensorFlow Datasets](https://www.tensorflow.org/datasets) 4.5.0
* [Tensorflow I/O](https://www.tensorflow.org/io) 0.21.0

We recommend that you create a separate environment for this project. These packages can be installed via `pip` or `conda` outside of this notebook or, alternatively, you can run:

In [ ]:
# OPTIONAL
import sys

!{sys.executable} -m pip install numpy==1.19.5 
!{sys.executable} -m pip install tensorflow==2.6.2 tensorflow-datasets==4.5.0  tensorflow-io==0.21.0
!{sys.executable} -m pip install keras-spiking==0.3.0

In case there are issues with running the [TensorFlow](https://www.tensorflow.org/) files on a GPU-equipped workstation, we recommend trying [TensorFlow](https://www.tensorflow.org/)
nightly:

In [ ]:
# OPTIONAL
!{sys.executable} -m pip install tf-nightly tensorflow-nightly-io

## Imports and hyperparameters

Let us import all the necessary packages:

In [ ]:
# -- Built-in modules -- #
import csv
import datetime

# -- Third-party modules -- #
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# -- Proprietary modules -- #
from auxiliary import plot_energy
from create_models import create_vgg16_model, create_spiking_vgg16_model
from dataloaders import load_data
from energy_estimations import energy_estimation
from utils import colour_str, DTStop

We also set the hyperparameters. They are defined as follows:

1. Dataset parameters
    * `DATASET` - chosen dataset; either `eurosat`, or `ucm`; one can also add  either `prewitt` or `sobel`, then the (normalised) Prewitt or Sobel transforms are applied to the input images, and  also, optionally `mask`, then the original images with those pixels, for  which the Prewitt or Sobel transform are zero, masked out are used as the input; if `sq` is added, then the transforms are squared (or, equivalently, the square root in the Prewitt or  obel transforms is not applied); so for example it can be `eurosat_prewitt_sq_mask` ot `ucm_sobel` etc.
  
2. ANN training parameters
    * `ANN_EPOCHS` - number of training epochs,
    * `ANN_BATCH_SIZE` - training batch size (per a replica),
    * `ANN_LEARNING_RATE`- learning rate,
    * `KERNEL_L2` - regularization L<sub>2</sub> parameter for the convolutional kernels,
    * `BIAS_L1` - regularization L<sub>1</sub> parameter for the convolutional biases,

3. SNN training parameters
    * `SNN_EPOCHS` - (starting) number of training epochs,
    * `SNN_BATCH_SIZE` - (starting) training batch size (per a replica),
    * `SNN_LEARNING_RATE`- (starting) learning rate,
    * `ITERATE` - boolean determining whether the training should be performed iteratively, doubling the number of timesteps, and halving the batch size, the number of epochs, and the learning rate at each iteration,
    * `TIMESTEPS` - number of the simulation timesteps,
    * `DT` - temporal resolution of timesteps; it is decreased during the training until it reaches the value of 1 ms (`DT_TARGET`),
    * `L2` - weight penalty for L<sub>2</sub> activity regularization of the spikes,
    * `LOWER_HZ` - lower frequency threshold for spiking rate regularization,
    * `UPPER_HZ` - upper frequency threshold for spiking rate regularization,
    * `TAU` - tau parameter for the low-pass filter.

4. Augmentation parameters 
    * `LOWER_ZOOM` - augmentation parameter; lower bound for a random zoom factor; must be positive,
    * `UPPER_ZOOM` - augmentation parameter; upper bound for a random zoom factor; must be bigger than `LOWER_ZOOM`.
    * `MAX_BRIGHTNESS_DELTA` - augmentation parameter; maximum brightness delta; must be a non-negative float,
    * `MAX_HUE_DELTA` - augmentation parameter; maximum hue delta; must be in the interval \[0, 0.5\],
    * `LOWER_CONTRAST` - augmentation parameter; lower bound for a random contrast factor; must be positive,
    * `UPPER_CONTRAST` - augmentation parameter; upper bound for a random contrast factor must be bigger than  LOWER_CONTRAST`,
    * `LOWER_SATURATION` - augmentation parameter; lower bound for a random saturation factor; must be positive,
    * `UPPER_SATURATION` - augmentation parameter; upper bound for a random saturation factor; must be bigger than `LOWER_SATURATION`.

5. Save paths
    * `ANN_MODEL_PATH` - path to where the trained ANN model is saved (if we want to avoid retraining) or will be saved (after training),
    * `SNN_MODELS_PATH` - path to where the trained SNN model is saved (if we want to avoid retraining) or will be saved (after training),

The default values of these parameters are the ones that empirically gave us the best test accuracy performance (91.43%) on [UC Merced](http://weegee.vision.ucmerced.edu/datasets/landuse.html) and (95.07%) on [EuroSat](https://github.com/phelber/EuroSAT) for the artificial neural network. But feel free to modify them if you want to experiment!

In [ ]:
# - Dataset parameters
DATASET = 'eurosat'
INPUT_SHAPE = (224, 224, 3) if 'ucm' in DATASET.lower() else (64, 64, 3)
NUM_CLASSES = 21 if 'ucm' in DATASET.lower() else 10

# - ANN training parameters
ANN_EPOCHS = 1000
ANN_BATCH_PER_REPLICA = 32
ANN_LR = .001
KERNEL_L2 = 1e-4
BIAS_L1 = 1e-5

# - SNN training parameters
SNN_EPOCHS = 16
SNN_BATCH_PER_REPLICA = 16
SNN_LR = 3e-5
ITERATE = False
TIMESTEPS = 32
DT = 1. # 1s
DT_TARGET = .001  # 1ms
L2 = 1e-9
LOWER_HZ = 10
UPPER_HZ = 20
TAU = .1

# - Augmentation parameters
LOWER_ZOOM = .95
UPPER_ZOOM = 1.05
MAX_BRIGHTNESS_DELTA = .2
MAX_HUE_DELTA = .1
LOWER_CONTRAST = .2
UPPER_CONTRAST = 1.8
LOWER_SATURATION =.9
UPPER_SATURATION = 1.1

# - Save paths
ANN_MODEL_PATH = 'ann_model.h5'
SNN_MODELS_PATH = 'snn_models'

We also set up (multiple) GPU training:

In [ ]:
# Strategy parameters (for multiple GPU training)
STRATEGY = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
NUM_DEVICES = STRATEGY.num_replicas_in_sync
print('Number of devices: {}'.format(NUM_DEVICES))

# Global batch sizes
ANN_BATCH_SIZE = ANN_BATCH_PER_REPLICA * NUM_DEVICES
SNN_BATCH_SIZE = SNN_BATCH_PER_REPLICA * NUM_DEVICES

## Training a VGG16-based model on EuroSAT RGB and UC Merced datasets
In the first part of the project we download either the [EuroSAT: Land Use and Land Cover Classification with Sentinel-2 Dataset](https://github.com/phelber/EuroSAT) (10 classes, 27000 examples) or the [UC Merced Land Use Dataset](http://weegee.vision.ucmerced.edu/datasets/landuse.html) (21 classes, 100 examples each). We slice it into the training, validation, and test sets using ratios 80%-10%-10%. We resize the [UC Merced](http://weegee.vision.ucmerced.edu/datasets/landuse.html) images
to (224, 224, 3) shape (to be compatible with the usual [VGG-16](https://neurohive.io/en/popular-networks/vgg16/) input size). We augment the training set using random dihedral group transformation, random crop, random brightness change,  random contrast change, random hue change, random saturation change, as specified in the augmentation hyperparameters above.

In [ ]:
# Display samples
download_name = 'uc_merced' if 'ucm' in DATASET else 'eurosat/rgb'
ds, info = tfds.load(download_name, split='train', with_info=True)
tfds.show_examples(ds, info)

# Augmentation
augmentation_parameters = {'lower_zoom': LOWER_ZOOM,
                           'upper_zoom': UPPER_ZOOM,
                           'max_brightness_delta': MAX_BRIGHTNESS_DELTA,
                           'max_hue_delta': MAX_HUE_DELTA,
                           'lower_contrast': LOWER_CONTRAST,
                           'upper_contrast': UPPER_CONTRAST,
                           'lower_saturation': LOWER_SATURATION,
                           'upper_saturation': UPPER_SATURATION}

# Load training, validation, and test data
train, val, test, _ = load_data(dataset=DATASET,
                                    input_size=INPUT_SHAPE[:-1],
                                    augmentation_parameters=augmentation_parameters,
                                    batch_size=ANN_BATCH_SIZE)

We use a modified version of the [VGG-16](https://neurohive.io/en/popular-networks/vgg16/) network trained on the [ImageNet](http://www.image-net.org/) dataset (parameters from the [Keras-TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16)) version to construct a classifier for this dataset. We replace the max pooling layers with average pooling layers, we remove the head of the network (all the
layers following the last pooling layers) and replace it with a global pooling layer, and a dense classifier without bias.

In [ ]:
with STRATEGY.scope():
    # - Create model
    model = create_vgg16_model(input_shape=INPUT_SHAPE,
                               kernel_l2=KERNEL_L2,
                               bias_l1=BIAS_L1,
                               num_classes=NUM_CLASSES,
                               remove_pooling=False,
                               use_dense_bias=False)

    # -- Compile the model
    model.compile(optimizer=tf.keras.optimizers.RMSprop(ANN_LR),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=tf.metrics.SparseCategoricalAccuracy())

# -- Show the summary of the model
model.summary()

The model is trained using the RMSprop optimizer, using early stopping and reducing the learning rate on a plateau (by a factor of 10) if there is no significant improvement in the validation loss after 100 and 50 consecutive
epochs respectively. Optionally, L<sub>2</sub> and L<sub>1</sub> regularization is applied to convolutional kernels and biases, respectively, if it was specified in the hyperparameters above. We save the best model to `ANN_MODEL_FILEPATH`.

In [ ]:
# Train the model (OPTIONAL)
# Callbacks                                                              
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(verbose=True, patience=50),
             tf.keras.callbacks.EarlyStopping(patience=100),
             tf.keras.callbacks.ModelCheckpoint(filepath=ANN_MODEL_PATH, verbose=True, save_best_only=True)]

# Fit the model on the training data
model.fit(x=train, epochs=ANN_EPOCHS, validation_data=val, callbacks=callbacks)

And finally, we evaluate the model:

In [ ]:
# Load the best weights
model.load_weights(ANN_MODEL_PATH)

# Evaluate the model
loss, acc = model.evaluate(x=test)
print("Best model's accuracy:", colour_str(f'{acc:.2%}', 'green'))

## Training a spiking model

A [VGG-16](https://neurohive.io/en/popular-networks/vgg16/) -based classifiertrained on the [EuroSat](https://github.com/phelber/EuroSAT) or [UC Merced](http://weegee.vision.ucmerced.edu/datasets/landuse.html) datasets can be converted into a spiking neural network and trained using [KerasSpiking](https://www.nengo.ai/keras-spiking/). Before the training, the local average pooling layers are removed, and the preceding convolutions have their strides set to 2, and their weights appropriately adjusted for  onsistency. The ReLU activation functions are swapped with [spiking activations](https://www.nengo.ai/keras-spiking/reference.html?highlight=spiking#keras_spiking.SpikingActivation) followed by a [low-pass filter](https://www.nengo.ai/keras-spiking/reference.html?highlight=spiking#keras_spiking.Lowpass)

In [ ]:
with STRATEGY.scope():
    # A variable to be passed to the model as the simulation time resolution
    dt_var = tf.Variable(DT, aggregation=tf.VariableAggregation.MEAN)

    # Functions to monitor the variables
    def dt_monitor(y_true, y_pred):
        return dt_var.read_value()

    # Create a model
    model = create_spiking_vgg16_model(model_path=ANN_MODEL_PATH,
                                       input_shape=INPUT_SHAPE,
                                       dt=dt_var,
                                       l2=L2,
                                       lower_hz=LOWER_HZ,
                                       upper_hz=UPPER_HZ,
                                       tau=TAU,
                                       num_classes=NUM_CLASSES,
                                       spiking_aware_training=True)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.RMSprop(SNN_LR),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy(), dt_monitor])

# Show model's summary
model.summary()

Training can be performed iteratively, doubling the number of timesteps, and halving the batch size, the number of epochs, and the learning rate at each iteration. We also need to reload the data, since now they have a temporal dimension.

In [ ]:
exponent = int(np.log2(TIMESTEPS))
start = 0 if ITERATE else exponent
for n in range(start, exponent + 1):
    # Iterative hyperparameters
    timesteps = 2 ** n
    batch_size = 2 ** (exponent - n) * SNN_BATCH_SIZE
    lr = SNN_LR * 2 ** (exponent - n)
    tf.keras.backend.set_value(model.optimizer.learning_rate, lr)
    epochs = SNN_EPOCHS * 2 ** (exponent - n)

    # Load data
    train, val, test, _ = load_data(DATASET,
                                    input_size=INPUT_SHAPE[:-1],
                                    augmentation_parameters=augmentation_parameters,
                                    batch_size=batch_size,
                                    timesteps=timesteps)

    # Callbacks
    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(patience=epochs // 4, verbose=True),
                 tf.keras.callbacks.EarlyStopping(patience=epochs // 2, verbose=True)]

    if dt_var.value() > DT_TARGET:
        callbacks.append(DTStop(dt=dt_var, dt_min=DT_TARGET))

    # Print the training iteration parameters
    print(f"Starting the training for {colour_str(epochs, 'orange')} epoch(s),",
          f"with {colour_str(timesteps, 'orange')} timestep(s),",
          f"on batches of {colour_str(batch_size, 'orange')} example(s),",
          f"and the learning rate {colour_str(lr, 'orange')}.")

    # Train the model
    print('Commencing the training on iteration',
          colour_str(f'{n - start + 1}/{exponent + 1 - start}', 'orange') + '.')
    model.fit(x=train, epochs=epochs, validation_data=val, callbacks=callbacks)

    # Evaluate the model
    results = model.evaluate(x=test, batch_size=batch_size, verbose=True)
    try:
        loss, acc, dt_stop = results
    except ValueError:
        loss, acc = results
        dt_stop = DT_TARGET

    print("Model's accuracy:", colour_str(f'{acc:.2%}', 'green'))

    # New model to avoid serialization issues
    with STRATEGY.scope():
        new_model = create_spiking_vgg16_model(model_path=ANN_MODEL_PATH,
                                               input_shape=INPUT_SHAPE,
                                               dt=dt_stop,
                                               l2=L2,
                                               lower_hz=LOWER_HZ,
                                               upper_hz=UPPER_HZ,
                                               tau=TAU,
                                               num_classes=NUM_CLASSES,
                                               spiking_aware_training=True)

        # - Load weights (skipping dt)
        new_model.set_weights([w for w in model.get_weights() if w.shape != ()])

        # - Compile the model
        new_model.compile(optimizer=tf.keras.optimizers.RMSprop(SNN_LR),
                          loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=[tf.metrics.SparseCategoricalAccuracy()])

    # Save model filepath
    new_model.save(f"{SNN_MODELS_PATH}/{n}.h5")

    # We stop optimising dt here
    if dt_stop <= DT_TARGET:
        model = new_model

    del new_model

### Energy consumption estimation
We can now compare the estimated energy consumption of the ANN and SNN (both with local pooling layers replaced by doubly-strided convolutions) on representative hardware (standard and neuromorphic). The value is estimated by considering the energy for a MAC operation. Such energy (`E_per_mac`) is obtained through a maximum-likelihood estimation: `E_inf = E_per_mac * N_ops`, after [G. Benelli, G. Meoni, and L. Fanucci, *Low power keyword spotting algorithm for memory constrained embedded systems*](https://ieeexplore.ieee.org/abstract/document/8644728). The particular values for standard hardware come from [B. Degnan, B. Marr, and J. Hasler, *Assessing Trends in Performance per Watt for Signal Processing Applications*](https://ieeexplore.ieee.org/abstract/document/7054508), the vallues for SpiNNaker come from [S. Höppner, et al, *Dynamic Power Management for Neuromorphic Many-Core Systems*](https://ieeexplore.ieee.org/document/8701528), while the values for Loihi are from [M. Davies, et al, *Loihi: A Neuromorphic Manycore Processor with On-Chip Learning*](https://ieeexplore.ieee.org/document/8259423).

In [ ]:
ann_synop_energy_dict, ann_neuron_energy_dict, ann_total_energy_dict = energy_estimation(model,
                                                                                         spiking_model=False,
                                                                                         verbose=True)

For the spiking energy estimation, we need to reload the test data (as the energy usage will depend on it):

In [ ]:
_, _, test, info = load_data(dataset=DATASET,
                             input_size=INPUT_SHAPE[:-1],
                             batch_size=SNN_BATCH_SIZE,
                             timesteps=TIMESTEPS)

# Discard the labels to conserve the energy and have no inconsistencies in the synaptic energy estimation model
xtest = test.map(lambda x, y: x, num_parallel_calls=tf.data.experimental.AUTOTUNE)

And finally, we can estimate the values:

In [ ]:
snn_synop_energy_dict, snn_neuron_energy_dict, snn_total_energy_dict = energy_estimation(model,
                                                                                         x_test=test,
                                                                                         spiking_model=True,
                                                                                         device_list=['loihi',
                                                                                                      'spinnaker',
                                                                                                      'spinnaker2'],
                                                                                         n_timesteps=TIMESTEPS,
                                                                                         dt=dt_stop,
                                                                                         verbose=True)